In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import matplotlib.pyplot as plt
#______________________________________________
from PIL import Image
import cv2
import numpy as np
from IPython.display import display
from google.colab.patches import cv2_imshow
#_______________________________________________
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:

base_dir = '/content/drive/MyDrive'
train_dir = os.path.join(base_dir, 'Dataset')

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
#To train Model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator
)

model.save('/content/drive/MyDrive/img_classification_model2.h5')


In [ ]:
Directory= r'/content/drive/MyDrive/Test'
data=[]
for img_item in os.listdir(Directory):
  img_item=os.path.join(Directory,img_item)
  data.append(img_item)

RefImage= r'/content/drive/MyDrive/Check'
Ref=[]
for img_item in os.listdir(RefImage):
  img_item=os.path.join(RefImage,img_item)
  Ref.append(img_item)


In [ ]:
# Load the saved model
saved_model = tf.keras.models.load_model('/content/drive/MyDrive/img_classification_model2.h5')
index=0
for item in data:
  image = cv2.imread(item)
  img=image

  image= cv2.resize(image, (224, 224))

  # Convert the image to array and normalize
  image_array = img_to_array(image) / 255.0
  image_array = np.expand_dims(image_array, axis=0)

  # Make predictions
  predictions = saved_model.predict(image_array)
  label_index = np.argmax(predictions)
  class_labels = train_generator.class_indices

  # Get the predicted label
  predicted_label = [k for k, v in class_labels.items() if v == label_index][0]
  print('Predicted label:', predicted_label)

  if predicted_label=="advance":
    ref = cv2.imread(Ref[0])
  elif predicted_label=="expense":
    ref = cv2.imread(Ref[1])
  elif predicted_label=="internet":
    ref = cv2.imread(Ref[2])
  elif predicted_label=="bill":
    ref = cv2.imread(Ref[3])

  print("___________________________________")
  # Detect keypoints and extract descriptors using SIFT
  sift = cv2.xfeatures2d.SIFT_create()
  kp_img, des_img = sift.detectAndCompute(img, None)
  kp_ref, des_ref = sift.detectAndCompute(ref, None)

  # Match keypoints between the images
  bf = cv2.BFMatcher()
  matches = bf.knnMatch(des_img, des_ref, k=2)

  # Apply ratio test to filter out bad matches
  good_matches = []
  for m, n in matches:
      if m.distance < 0.75 * n.distance:
          good_matches.append(m)

  # Draw the matched keypoints
  img_match = cv2.drawMatches(img, kp_img, ref, kp_ref, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

  # Extract the matched keypoints' coordinates
  src_pts = np.float32([kp_img[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  dst_pts = np.float32([kp_ref[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

  # Calculate the homography matrix
  H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

  # Apply the homography transformation to the image
  warped_img = cv2.warpPerspective(img, H, (ref.shape[1], ref.shape[0]))

  # display(warped_img)
  # Display the original image, reference image, and warped image
  index+=1
  resized_img = cv2.resize(warped_img, (300, 200))
  cv2_imshow(resized_img)
  if predicted_label=="advance":
    folder_path = '/content/drive/MyDrive/Sort/advance/'
  elif predicted_label=="expense":
    folder_path = '/content/drive/MyDrive/Sort/expense/'
  elif predicted_label=="internet":
    folder_path = '/content/drive/MyDrive/Sort/internet/'
  elif predicted_label=="bill":
    folder_path = '/content/drive/MyDrive/Sort/bill/'

  cv2.imwrite(folder_path + f'{index}.png', warped_img)

